<a href="https://colab.research.google.com/github/rubbybbs/ProbML-book-solution/blob/main/ch01/1.4.3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 习题 1.4.3 <a href="https://colab.research.google.com/github/rubbybbs/ProbML-book-solution/blob/main/ch01/1.4.3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

编程实现K-近邻算法，并在MNIST手写字符识别数据集上进行实验：

下载并预处理MNIST手写字符识别数据集

In [1]:
import numpy as np
import urllib.request
import gzip
import pandas as pd

np.random.seed(2023)

# Download the MNIST dataset
urllib.request.urlretrieve(
    "http://yann.lecun.com/exdb/mnist/train-images-idx3-ubyte.gz", 
    "train-images.gz")
urllib.request.urlretrieve(
    "http://yann.lecun.com/exdb/mnist/train-labels-idx1-ubyte.gz",
    "train-labels.gz")
urllib.request.urlretrieve(
    "http://yann.lecun.com/exdb/mnist/t10k-images-idx3-ubyte.gz", 
    "test-images.gz")
urllib.request.urlretrieve(
    "http://yann.lecun.com/exdb/mnist/t10k-labels-idx1-ubyte.gz",
    "test-labels.gz")

# Load the training data
with gzip.open('train-images.gz', 'rb') as f:
    train_images = np.frombuffer(f.read(), np.uint8, offset=16).reshape(-1, 28*28)
with gzip.open('train-labels.gz', 'rb') as f:
    train_labels = np.frombuffer(f.read(), np.uint8, offset=8)

# Load the test data
with gzip.open('test-images.gz', 'rb') as f:
    test_images = np.frombuffer(f.read(), np.uint8, offset=16).reshape(-1, 28*28)
with gzip.open('test-labels.gz', 'rb') as f:
    test_labels = np.frombuffer(f.read(), np.uint8, offset=8)

# Normalize the data
train_images = train_images.astype(np.float32) / 255.0
test_images = test_images.astype(np.float32) / 255.0

# Generate dataset
n_training_per_class = 1000
X_train = []
y_train = []
for i in range(n_training_per_class):
    class_indices = np.where(train_labels == i)[0][:n_training_per_class]
    X_train.append(train_images[class_indices])
    y_train.append(np.full((n_training_per_class,), i))
X_train = np.concatenate(X_train, axis=0)
y_train = np.concatenate(y_train, axis=0)

n_test = 1000
indices_test = np.random.choice(np.arange(len(test_images)), size=n_test, replace=False)
X_test = test_images[indices_test]
y_test = test_labels[indices_test]

实现K-近邻算法

In [5]:
def l2_distance(x1, x2, axis=-1):
    return np.sqrt(np.sum((x1 - x2) ** 2), axis=axis)

def l1_distance(x1, x2, axis=-1):
    return np.sum(np.abs(x1 - x2), axis=axis)

class KNN:
    def __init__(self, X_train, y_train, k=5, distance_func=l1_distance):
        self.X_train = X_train
        self.y_train = y_train
        self.k = k
        self.distance_func = distance_func

    def predict(self, X_test):
        y_pred = np.zeros(len(X_test), dtype=self.y_train.dtype)
        for i, x in enumerate(X_test):
            distances = self.distance_func(x, self.X_train, axis=-1)
            idx = np.argsort(distances)[:self.k]
            k_nearest_classes = self.y_train[idx]
            y_pred[i] = np.bincount(k_nearest_classes).argmax()
        return y_pred


(1) 分析比较不同$K$取值（如$K=1, 3,5,7$)对分类正确率的影响

In [6]:
k_list = [1, 3, 5, 7]
acc_list = []
for k in k_list:
    knn = KNN(X_train, y_train, k=k)
    y_pred = knn.predict(X_test)
    acc_list.append(np.sum(y_pred == y_test) / len(y_pred))

df_result = pd.DataFrame()
df_result['K value'] = k_list
df_result['Test Acc'] = acc_list
df_result

,K value,Test Acc
0,1,0.933
1,3,0.934
2,5,0.934
3,7,0.936


(2) 